# CNMOW evaluation results

In [ ]:
import json
import os
from os.path import realpath, dirname, join, splitext
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

ROOT = './data/'
OUTPUT_DIR = join(ROOT, 'plots')
os.makedirs(OUTPUT_DIR, exist_ok=True)
baselines = [
    'cbow-784-10p',
    'cmow-784-10p',
    'hybrid-800-10p',   
]

probing_tasks = sorted(['Tense', 'SubjNumber', 'BigramShift', 'CoordinationInversion', 'ObjNumber', 
                 'Length', 'Depth', 'OddManOut', ])
downstream_tasks = sorted(['STS15', 'STS16', 
                            # Just for convenience
                            'STS15\nspearman', 'STS16\nspearman',
                            'SICKEntailment', 
                            'MRPC', 'TREC', ])
all_tasks = probing_tasks + downstream_tasks

In [ ]:
sns.set_style("whitegrid")
sns.set_palette("PuBuGn_d")

## Available models

In [ ]:
available = []
n_models = 0
n_evals = 0
for f in os.listdir(ROOT):
    if f.startswith('model-'):
        n_models += 1
        available.append(f)
    if f.startswith('evaluation-'):
        n_evals += 1
        available.append(f)

print('Found {} models, {} evaluations:\n'.format(n_models, n_evals))
available = sorted(available)
for f in available:
    print("'{}',".format(f))

## Benchmark performance

### Parsing code

In [ ]:
def extract_values(model_name, row):
    extracted = pd.DataFrame(columns=['Model', 'Benchmark', 'Score'])
    s = pd.Series(index=extracted.columns)
    for k, entry in row.items():
        s['Model'] = model_name
        
        if isinstance(entry, str):
            if entry.lower() == 'none':
                l = (np.nan, np.nan)
                print('Warning: model {} has result "{}" for benchmark {}'.format(model_name, entry, k), 
                      file=sys.stderr)
            else:
                try:
                    l = json.loads(entry.replace('(', '[').replace(')', ']'))
                except json.JSONDecodeError as e:
                    print('For model {}, could not extract pair from entry: {}'.format(
                        model_name, entry))
                    raise e
            # Pairs of numbers represent (spearman, pearson)
            s['Benchmark'] = k + '\nspearman'
            s['Score'] = float(l[0])
            extracted = extracted.append(s, ignore_index=True)
            #s['Benchmark'] = k + ' - pearson'
            #s['Score'] = float(l[1])
            #extracted = extracted.append(s, ignore_index=True)
        else:
            s['Benchmark'] = k
            s['Score'] = entry
            extracted = extracted.append(s, ignore_index=True)
    return extracted

def extract_model_name(row):
    import json
    n = row['outputmodelname']
    try:
        s = json.loads(n.replace("'", '"'))
        return s[0]
    except ValueError as e:
        print(e)
        pass
    return n

def parse_results(eval_filenames):
    extracted = None
    model_names = {}
    for fname in eval_filenames:
        df = pd.read_csv(fname, sep=';', header=0)
        for i, row in df.iterrows():
            evals = json.loads(row['downstream_tasks'].replace("'", '"'))
            # There's one model per row
            model_name = extract_model_name(row)
            if model_name in model_names:
                print('Model name "{}" seen in two files: \n- {}\n- {}'.format(
                    model_name, fname, model_names[model_name]
                ), file=sys.stderr)
            model_names[model_name] = fname
            ee = extract_values(model_name, row[evals])
            if extracted is None:
                extracted = ee
            else:
                extracted = extracted.append(ee)
    return extracted

### Plotting code

In [ ]:
def plot_evaluation_results(eval_filenames, baselines=None, filename='comparison.pdf', palette=None):
    extracted = parse_results(eval_filenames)
          
    fig, ax = plt.subplots(2, 1, figsize=(16, 2*7))
    for i, coeff_metrics in enumerate([False, True]):
        mask = extracted['Benchmark'].str.contains("spearman|pearson")
        if not coeff_metrics:
            mask = ~mask
        selected = extracted[mask]
        
        # Keep desired ordering of tasks (probing, then downstream)
        selected_benchmarks = selected['Benchmark'].unique()
        benchmarks = [t for t in all_tasks if t in selected_benchmarks]
        assert set(benchmarks) == set(selected_benchmarks), '{} vs {}'.format(benchmarks, selected_benchmarks)
        
        sns.barplot(x='Benchmark', y='Score', hue='Model', data=selected, order=benchmarks, ax=ax[i],
                    palette=palette)
        ax[i].legend(loc=2, bbox_to_anchor=(1,1))
        ax[i].set_title('Evaluation results')
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, horizontalalignment='right')
        
        if baselines is not None:
            baseline_mask = selected['Model'].str.match('|'.join(baselines))
            baseline_results = selected[baseline_mask]
            # Draw the high-line for each metric (whichever baseline is best for that metric)
            length = (ax[i].get_xlim()[1] - ax[i].get_xlim()[0]) / len(benchmarks)
            for j, bench in enumerate(benchmarks):
                max_val = baseline_results[baseline_results['Benchmark'] == bench]['Score'].max()
                ax[i].plot([length*(j-0.5), length*(j+0.5)], [max_val, max_val], '-k', alpha=0.35)
        
    plt.tight_layout()
    out = join(OUTPUT_DIR, filename)
    plt.savefig(out, bbox_inches='tight', dpi=128)
    plt.savefig(splitext(out)[0] + '.png', bbox_inches='tight', dpi=200)

### All available evaluations

In [ ]:
# model-cnmow3c-784-10p, model-cnmow4c-784-10p, 
# model-cnmow7b-784-10p,
# model-cnmow1-hybrid-800-10p model-cnmow2-hybrid-800-10p
all_sources = [
    'evaluation-cbow-784-10p',
    'evaluation-cmow-784-10p',
    'evaluation-hybrid-800-10p',
    
    'evaluation-cnmow1-784-10p',
    'evaluation-cnmow1-hybrid-800-10p',
    'evaluation-cnmow1b-784-10p',
    'evaluation-cnmow2-784-10p',
    'evaluation-cnmow2-hybrid-800-10p',
    'evaluation-cnmow2b-784-10p',
    'evaluation-cnmow3-784-10p',
    'evaluation-cnmow3-hybrid-800-10p',
    'evaluation-cnmow3c-784-10p',
    'evaluation-cnmow4-784-10p',
    'evaluation-cnmow4-hybrid-800-10p',
    'evaluation-cnmow4c-784-10p',
    'evaluation-cnmow5-784-10p',
    'evaluation-cnmow5-hybrid-800-10p',
    'evaluation-cnmow6-784-10p',
    'evaluation-cnmow6-hybrid-800-10p',
    'evaluation-cnmow6b-784-10p',
    'evaluation-cnmow7-784-10p',
    'evaluation-cnmow7-hybrid-800-10p',
    'evaluation-cnmow7b-784-10p',
    'evaluation-cnmow8-784-10p',
    'evaluation-cnmow8-hybrid-800-10p',
    'evaluation-cnmow9-784-10p',
    'evaluation-cnmow9-hybrid-800-10p',
]
plot_evaluation_results([join(ROOT, source, 'evaluation.csv') for source in all_sources], baselines=baselines,
                        filename='comparison-all.pdf')

In [ ]:
sns.barplot()

### ReLU vs Sigmoid

In [ ]:
sources = [
    'evaluation-cnmow1-784-10p',
    'evaluation-cnmow1b-784-10p',
    'evaluation-cnmow2-784-10p',
    'evaluation-cnmow2b-784-10p',
    'evaluation-cnmow6-784-10p',
    'evaluation-cnmow6b-784-10p',
    'evaluation-cnmow7-784-10p',
    'evaluation-cnmow7b-784-10p',
]
plot_evaluation_results([join(ROOT, source, 'evaluation.csv') for source in sources], baselines=baselines,
                        filename='comparison-nonlinearity.pdf', 
                        palette=[sns.color_palette()[0], sns.color_palette()[-1]])

Conclusion: ReLU is better, except for some tasks.

### Effect of simply adding a nonlinearity to CMOW

In [ ]:
sources = [
    'evaluation-cmow-784-10p',
    'evaluation-cnmow1-784-10p',
    'evaluation-cnmow2-784-10p',
]
plot_evaluation_results([join(ROOT, source, 'evaluation.csv') for source in sources], baselines=baselines,
                        filename='comparison-cmow-cnmow.pdf', 
                        palette=[sns.color_palette()[0], sns.color_palette()[-2], sns.color_palette()[-1]])

### Hybrid only

In [ ]:
sources = [
    'evaluation-hybrid-800-10p',
    
    'evaluation-cnmow1-hybrid-800-10p',
    'evaluation-cnmow2-hybrid-800-10p',
    'evaluation-cnmow3-hybrid-800-10p',
    'evaluation-cnmow4-hybrid-800-10p',
    'evaluation-cnmow5-hybrid-800-10p',
    'evaluation-cnmow6-hybrid-800-10p',
    'evaluation-cnmow7-hybrid-800-10p',
    'evaluation-cnmow8-hybrid-800-10p',
    'evaluation-cnmow9-hybrid-800-10p',
]
plot_evaluation_results([join(ROOT, source, 'evaluation.csv') for source in sources], baselines=baselines,
                        filename='comparison-hybrid.pdf', 
                        palette=[sns.color_palette()[0], sns.color_palette()[-2], sns.color_palette()[-1]])

### Models that perform better

In [ ]:
df = parse_results([join(ROOT, source, 'evaluation.csv') for source in all_sources])
better_models = {}
unique_tasks = df['Benchmark'].unique()
print('There are {} tasks.'.format(len(unique_tasks)))
for task in unique_tasks:
    subset = df[df['Benchmark'] == task]
    baseline_mask = subset['Model'].str.match('|'.join(baselines))
    baseline_max = subset[baseline_mask]['Score'].max()
    better_mask = subset['Score'] > baseline_max
    for j, row in subset[better_mask].iterrows():
        name = row['Model']
        if name not in better_models:
            better_models[name] = []
        tsk = task.replace('\n', ' ')
        better_models[name].append(tsk)
        print('- Model {} is better at task {}:  {}  >  {}'.format(
            name, tsk, row['Score'], baseline_max
        ))
        
print('\n Summary:')
for model, tasks in better_models.items():
    print('- Model {} is better at {} tasks ({})'.format(model, len(tasks), ','.join(tasks)))

In [ ]:
# TODO: actually pick the best models
sources = [
    'evaluation-cbow-784-10p',
    'evaluation-cmow-784-10p',
    'evaluation-hybrid-800-10p',
    
    # 'evaluation-cnmow1-784-10p',
    # 'evaluation-cnmow1b-784-10p',
    # 'evaluation-cnmow2-784-10p',
    # 'evaluation-cnmow2b-784-10p',
    # 'evaluation-cnmow3-hybrid-800-10p',
    # 'evaluation-cnmow6-hybrid-800-10p',
    'evaluation-cnmow2-784-10p',
    'evaluation-cnmow2-hybrid-800-10p',
]
plot_evaluation_results([join(ROOT, source, 'evaluation.csv') for source in sources], baselines=baselines,
                        filename='comparison-best.pdf')

## Training times

In [ ]:
import csv

def parse_training_metadata(filenames):
    if not isinstance(filenames, (list, tuple)):
        filenames = [filenames]
        
    df = pd.DataFrame(columns=['Variant name', 'Docs count', 'Training time', 'Epoch count'])
    
    for fname in filenames:
        try:
            parsed = pd.read_csv(fname, sep=',', usecols=(0,1,2,3), header=0, index_col=False, 
                                 squeeze=True, skipinitialspace=True)
        except Exception as e:
            print('Could not parse metadata from: {}'.format(fname), file=sys.stderr)
            raise e
        df = df.append(parsed, ignore_index=True)
        
    # Mark invalid data with NaN
    df['Training time'] = df['Training time'].where(lambda v: v > 1)
    df['Epoch count'] = df['Epoch count'].where(lambda v: v > 1)
    df['Sentence / second'] = df['Docs count'] * df['Epoch count'] / df['Training time']
    return df
        
    
def plot_training_metadata(df, baselines=None, filename='training-times.pdf'):
    fig, ax = plt.subplots(3, 1, figsize=(16, 3*7))
    for i, metric in enumerate(['Training time', 'Epoch count', 'Sentence / second']):
        sns.barplot(x='Variant name', y=metric, data=df, ax=ax[i])
        #ax[i].set_title(metric)
        ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=45, horizontalalignment='right')
        ax[i].set_xlabel('')
        
        if i == 0:
            ax[i].set_ylim((0, 25000))
        
    plt.tight_layout()
    out = join(OUTPUT_DIR, filename)
    plt.savefig(out, bbox_inches='tight', dpi=128)
    plt.savefig(splitext(out)[0] + '.png', bbox_inches='tight', dpi=200)

In [ ]:
models_trained = [
    'model-cbow-784-10p',
    'model-cmow-784-10p',
    'model-hybrid-800-10p',
    'model-cnmow1-784-10p',
    'model-cnmow1b-784-10p',
    'model-cnmow2-784-10p',
    'model-cnmow2b-784-10p',
    'model-cnmow3-784-10p',
    'model-cnmow3-hybrid-800-10p',
    'model-cnmow4-784-10p',
    'model-cnmow4-hybrid-800-10p',
    'model-cnmow5-784-10p',
    'model-cnmow5-hybrid-800-10p',
    'model-cnmow6-784-10p',
    'model-cnmow6-hybrid-800-10p',
    'model-cnmow6b-784-10p',
    'model-cnmow7-784-10p',
    'model-cnmow7-hybrid-800-10p',
    'model-cnmow8-784-10p',
    'model-cnmow8-hybrid-800-10p',
    'model-cnmow9-784-10p',
    'model-cnmow9-hybrid-800-10p',
]

df = parse_training_metadata([join(ROOT, source, 'metadata.csv') for source in models_trained])
plot_training_metadata(df)

# Hyperparameter tuning

In [ ]:
data_alpha = np.genfromtxt('EE_3_val_loss.csv',delimiter=',')

In [ ]:
alphas = [1,2,4,8,16,32,64]

In [ ]:
plt.errorbar(alphas,np.mean(data_alpha,axis=1),yerr=(1/np.sqrt(10))*np.std(data_alpha,axis=1))
plt.xlabel('alpha values')
plt.ylabel('validation loss')
plt.show()

In [ ]:
lamb_1 = np.genfromtxt('lambV2_val_loss.csv',delimiter=',')
lamb_2 = np.genfromtxt('lambV2_2_val_loss.csv',delimiter=',')
lamb_3 = np.genfromtxt('lambV2_3_val_loss.csv',delimiter=',')

In [ ]:
lambdas = [0,0.25,0.5,0.75,1.0]

In [ ]:
plt.errorbar(lambdas,np.mean(lamb_1,axis=1),yerr=(1.96/np.sqrt(10))*np.std(lamb_1,axis=1))
plt.xlabel('lambda values')
plt.ylabel('validation loss')
plt.show()

In [ ]:
plt.errorbar(lambdas,np.mean(lamb_2,axis=1),yerr=(1.96/np.sqrt(10))*np.std(lamb_2,axis=1))
plt.xlabel('lambda values')
plt.ylabel('validation loss')
plt.show()

In [ ]:
plt.errorbar(lambdas,np.mean(lamb_3,axis=1),yerr=(1.96/np.sqrt(10))*np.std(lamb_3,axis=1))
plt.xlabel('lambda values')
plt.ylabel('validation loss')
plt.show()

In [ ]:
np.mean(lamb_3,axis=1)